<a href="https://colab.research.google.com/github/LucasFDutra/machine_learning_studies/blob/master/machine_learning_studies/Music_Generation_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libs


In [1]:
# Import Tensorflow 2.0
%tensorflow_version 2.x
import tensorflow as tf 

# Download and import the MIT 6.S191 package
!pip install mitdeeplearning
import mitdeeplearning as mdl

# Import all remaining packages
import numpy as np
assert len(tf.config.list_physical_devices('GPU')) > 0

     |████████████████████████████████| 2.1MB 4.5MB/s 
  Created wheel for mitdeeplearning: filename=mitdeeplearning-0.1.2-cp36-none-any.whl size=2114585 sha256=7c6eebff0f6ce76d35a662b20fd8427abe0296db1705b47170c761d9b6a42d9c
  Stored in directory: /root/.cache/pip/wheels/27/e1/73/5f01c787621d8a3c857f59876c79e304b9b64db9ff5bd61b74
Successfully built mitdeeplearning


# Dataset

Vou pegar os dados com o auxilio da biblioteca do MIT


In [2]:
songs = mdl.lab1.load_training_data()

Found 816 songs in text


In [5]:
songs[0]

'X:2\nT:An Buachaill Dreoite\nZ: id:dc-hornpipe-2\nM:C|\nL:1/8\nK:G Major\nGF|DGGB d2GB|d2GF Gc (3AGF|DGGB d2GB|dBcA F2GF|!\nDGGB d2GF|DGGF G2Ge|fgaf gbag|fdcA G2:|!\nGA|B2BG c2cA|d2GF G2GA|B2BG c2cA|d2DE F2GA|!\nB2BG c2cA|d^cde f2 (3def|g2gf gbag|fdcA G2:|!'

# Criando um vocabulário

Verificar os caracteres únicos

In [9]:
songs_joined = "\n\n".join(songs)

vocab = sorted(set(songs_joined))

print ('Existem ', len(vocab), ' caracteres diferentes no vocabulário')

Existem  83  caracteres diferentes no vocabulário


# Criando um dicionário de dados

Vou pegar todos os carateres presentes no dataset e atribuir um valor numérico para cada um.

ex: a = 0, b = 1, ...

E também criarei um meio de voltarmos, colocando os caracteres em um array

ex: `['a', 'b']`

Assim saberemos que o caractere representado pelo número 0 é o 'a', pois ele ocupa a posição zero no array

In [27]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

print('em char2idx: ', list(char2idx.items())[0])

em char2idx:  ('\n', 0)


# Convertendo o array de letras nos numeros correspondentes do nosso diconário de dados

In [29]:
def vectorize_string(string):
    vectorized_output = np.array([char2idx[char] for char in string])
    return vectorized_output

vectorized_songs = vectorize_string(songs_joined)

vectorized_songs

array([49, 22, 14, ..., 22, 82,  2])

# Definir função para pegar batchs de dados
```python
vectorized_songs = [1,2,3,4]

seq_length = 3

x = [1,2,3]
y = [2,3,4]
```

In [42]:
def get_batch(vectorized_songs, seq_length, batch_size):
    n = len(vectorized_songs) - 1
    idx = np.random.choice(n - seq_length, batch_size)

    input_batch = [vectorized_songs[i : seq_length+i] for i in idx]
    output_batch = [vectorized_songs[i+1 : seq_length+i+1] for i in idx]

    x_batch = np.reshape(input_batch, [batch_size, seq_length])
    y_batch = np.reshape(output_batch, [batch_size, seq_length])

    return x_batch, y_batch

In [43]:
x_batch, y_batch = get_batch(vectorized_songs, seq_length=5, batch_size=1)

print(x_batch, y_batch)

[[82 61 56 56 61]] [[61 56 56 61  1]]


# Construindo o modelo

uma camada `tf.keras.layers.Embedding`

uma camada `tf.keras.layers.LSTM`

uma camada `tf.keras.layers.Dense